# Full PyRadiomics-mpReview Repeatability Study
Analyse pyRadiomics features on mpReview prostate data for repeatability.

Code cells in which you are epxected to provide user input/set parameters to customize the anylsis are headed by "*__User Input__*" (of course you are free to customize any part of the code, this is just to help you find the parameters which are most intersting for most users).

NOTE: There are some asserts included in the code to make sure some basic aspects of the data handling turned out as expected. If any of these fail, smth most likely went wrong with the data processing (or some crucial thing about the data is different than expected when this code was written).

## Preparation

In [1]:
import os
import pandas as pd
# tell pandas the display width (more ore less), since it cannot autodetect in a browser, 
# lower this if it is too high for your display (and you encounter ugly wrapping effects)
pd.set_option('display.width', 120)
# Disable warnings about changes to a df copy not making it back to the original df
pd.options.mode.chained_assignment = None  # default='warn'

# This line tells the notebook to show plots inside of the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sb
import numpy as np

##### User Input - Set the CSV files containing the extracted pyRadiomics features:

In [2]:
BASE_FILE_NAMES = [#"pyRadiomicsFeatures_FullStudySettings_2D_bin5",
                   "pyRadiomicsFeatures_FullStudySettings_2D_bin10",
                   "pyRadiomicsFeatures_FullStudySettings_2D_bin15",
                   "pyRadiomicsFeatures_FullStudySettings_2D_bin20",
                   "pyRadiomicsFeatures_FullStudySettings_2D_bin40",
                   #"pyRadiomicsFeatures_FullStudySettings_3D_bin5",
                   "pyRadiomicsFeatures_FullStudySettings_3D_bin10",
                   "pyRadiomicsFeatures_FullStudySettings_3D_bin15",
                   "pyRadiomicsFeatures_FullStudySettings_3D_bin20",
                   "pyRadiomicsFeatures_FullStudySettings_3D_bin40",
                   #"pyRadiomicsFeatures_FullStudySettings_noNormalization_2D_bin5",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_2D_bin10",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_2D_bin15",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_2D_bin20",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_2D_bin40",
                   #"pyRadiomicsFeatures_FullStudySettings_noNormalization_3D_bin5",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_3D_bin10",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_3D_bin15",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_3D_bin20",
                   "pyRadiomicsFeatures_FullStudySettings_noNormalization_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_2D_bin40",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_2D_bin10",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_2D_bin15",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_2D_bin20",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_2D_bin40",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_2D_bin10",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_2D_bin15",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_2D_bin20",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_3D_bin10",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_3D_bin15",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_3D_bin20",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_3D_bin10",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_3D_bin15",
                   "pyRadiomicsFeatures_biasCorrectedT2AX_FullStudySettings_noNormalization_3D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_2D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_2D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_2D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_2D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_2D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_2D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_2D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_2D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_3D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_3D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_3D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_3D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_3D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+MuscleRefNormT2AX_FullStudySettings_noNormalization_3D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_2D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_2D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_2D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_2D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_2D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_2D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_2D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_2D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_3D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_3D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_3D_bin20",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_3D_bin40",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_3D_bin10",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_3D_bin15",
                   "pyRadiomicsFeatures_biasCorrected+TP2RegisteredT2AX_FullStudySettings_noNormalization_3D_bin20"]

featuresFileNames = [os.path.join(os.getcwd(), "EvalData", b + ".csv") for b in BASE_FILE_NAMES]
#print(featuresFileNames)

Load the feature data and add some more derived meta information (which will be convenient later)

In [3]:
dfFeatures = None
for i, fileName in enumerate(featuresFileNames):
  dfImport = pd.read_csv(fileName, sep = ";")
  assert "_bin" in fileName
  dfImport["binwidth"] = int(fileName[fileName.find("_bin")+4:].split(".")[0])
  assert "_3D" in fileName or "_2D" in fileName
  dfImport["dim"] = 3 if "_3D" in fileName else 2
  dfImport["normalized"] = False if "_noNormalization" in fileName else True
  baseImageType = "ORIG"
  if "_biasCorrectedT2AX" in fileName:
    baseImageType = "BC"
  elif "_biasCorrected+MuscleRefNormT2AX" in fileName:
    baseImageType = "BC+MRN"
  elif "_biasCorrected+TP2RegisteredT2AX" in fileName:
    baseImageType = "BC+REG"
  dfImport["baseImageType"] = baseImageType
  assert (dfImport.shape[0] == 102) or (dfImport.shape[0] == 22) or (dfImport.shape[0] == 326)
  if i == 0:
    dfFeatures = dfImport
  else:
    dfFeatures = pd.concat([dfFeatures, dfImport], ignore_index=True)

dfFeatures["series"] = dfFeatures["series"].astype(str)
dfFeatures["date"] = dfFeatures.apply(lambda row: row.study.split("_")[1], axis = 1)
dfFeatures["date"] = pd.to_numeric(dfFeatures["date"])
dfFeatures["patient"] = dfFeatures.apply(lambda row: row.study.split("_")[0], axis = 1)

#assert dfFeatures.shape[0] == len(featuresFileNames) * 102  #326
#display(dfFeatures[(dfFeatures.patient == "PCAMPMRI-00775") & (dfFeatures.segmentedStructure == "NormalROI_PZ_1")])

D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Merge the data in such a way that ascociated measurments for both timepoints for each patient are combined in one row with "\_t1" indicating measurements from the earlier (first) timepoint and "\_t2" indicating measurements from the later (second) timepoint.

In [4]:
dfFeaturesJointTP = pd.merge(dfFeatures, dfFeatures, how = "inner", suffixes = ("_t1", "_t2"),
                             on = ["patient", "canonicalType", "segmentedStructure", "binwidth", "dim", "normalized", "baseImageType"])
dfFeaturesJointTP = dfFeaturesJointTP[(dfFeaturesJointTP.date_t1 < dfFeaturesJointTP.date_t2)]
print(dfFeaturesJointTP.shape)

# exclude PCAMPMRI-00775 (case 1) ADC1400, because the 2nd timepoint image is not ok
dfFeaturesJointTP = dfFeaturesJointTP[~((dfFeaturesJointTP.patient == "PCAMPMRI-00775") &
                                        (dfFeaturesJointTP.canonicalType == "ADC1400"))]

#dfFeaturesJointTP.to_csv("D:/pyRadiomicsFeaturesJointTimepoints.csv", sep = ";", index = False)

#assert dfFeaturesJointTP.shape[0] == len(featuresFileNames) * 48   #160

#display(dfFeaturesJointTP[["series_t1", "series_t2", "patient", "canonicalType", "segmentedStructure", "binwidth", "dim", "normalized"]])

(4272, 3083)


## Compute Metrics

In [5]:
selectedFeatures = [c[0:-3] for c in dfFeaturesJointTP.columns if any(xc in c for xc in ["original", 
                                                                                         "log-sigma", 
                                                                                         "wavelet",
                                                                                         "square",
                                                                                         "squareroot",
                                                                                         "logarithm",
                                                                                         "exponential"
                                                                                        ])]


selectedFeatures = set(selectedFeatures)
#selectedFeatures = ["original_shape_Volume"]
#print("No. Selected Features:", len(selectedFeatures), "\nSelected Features:", selectedFeatures)

# Select image types: choose from ["ADC1400", "SUB", "T2AX"]
selectedTypes = ["ADC1400", "SUB", "T2AX"]

# Select segmented structures: choose from ["WholeGland", "PeripheralZone", "TumorROI_PZ_1", "NormalROI_PZ_1"]
selectedStructures = ["WholeGland", "PeripheralZone", "TumorROI_PZ_1"]

Get metrics for selected data, add some meta data to the metrics and create a unsorted and a sorted DataFrame from the metrics.

In [6]:
import statsUtils.metrics as sm

metrics = []
for ctype in selectedTypes:
  for structure in selectedStructures:
    for baseImageType in dfFeaturesJointTP["baseImageType"].unique().tolist():
      for binwidth in dfFeaturesJointTP["binwidth"].unique().tolist():
        for dim in dfFeaturesJointTP["dim"].unique().tolist():
          for normalized in dfFeaturesJointTP["normalized"].unique().tolist():
            dfSelection = dfFeaturesJointTP[(dfFeaturesJointTP.canonicalType == ctype) & 
                                            (dfFeaturesJointTP.segmentedStructure == structure) &
                                            (dfFeaturesJointTP.baseImageType == baseImageType) &
                                            (dfFeaturesJointTP.binwidth == binwidth) &
                                            (dfFeaturesJointTP.dim == dim) &
                                            (dfFeaturesJointTP.normalized == normalized)]
            if dfSelection.shape[0] > 0:
              for feature in selectedFeatures:
                f1 = feature + "_t1"
                f2 = feature + "_t2"
                t1Data = dfSelection[f1]
                t2Data = dfSelection[f2]
                
                assert t1Data.shape[0] == t2Data.shape[0]
                expectedCases = 15
                if structure == "TumorROI_PZ_1":
                  expectedCases = 11
                if ctype == "ADC1400":
                  expectedCases -= 1
                assert t1Data.shape[0] == expectedCases
                
                m = sm.getMetrics(t1Data, t2Data)
                m["featureName"] = feature
                m["featureGroup"] = feature.split("_")[1]
                m["featureBaseName"] = feature.rsplit("_", maxsplit = 1)[1]
                m["featureGroupAndBaseName"] = feature.split("_", maxsplit = 1)[1]
                m["structure"] = structure
                m["ctype"] = ctype
                m["filter"] = feature.split("_")[0]
                filterGroup = (feature.split("_")[0]).split("-")[0]
                if filterGroup not in ["wavelet", "log", "original"]:
                  filterGroup = "singlepix"
                if filterGroup == "original":
                  filterGroup = "a_original"
                m["filterGroup"] = filterGroup
                m["binwidth"] = binwidth
                m["dim"] = dim
                m["normalized"] = normalized
                m["baseImageType"] = baseImageType
                metrics.append(m)
      
dfMetrics = pd.DataFrame(metrics)
#print(dfMetrics.sort_values(by = "structure")[["ctype", "structure", "icc", "rc", "rcp", "meanOfAbsDiff", "meanOfAbsDiffPercent"]])
#print(dfMetrics["filter"].unique())

dfSortedMetrics = dfMetrics.sort_values(by = "icc", ascending = False)
#print(dfSortedMetrics.head(5))


D:\miniconda3\envs\QINProst\lib\site-packages\numpy\core\_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
D:\Dev\PCrash\QIN-PROSTATE-Repeatability-Radiomics\statsUtils\metrics.py:23: RuntimeWarning: invalid value encountered in subtract
  diff = data1 - data2
D:\miniconda3\envs\QINProst\lib\site-packages\numpy\core\_methods.py:75: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
D:\Dev\PCrash\QIN-PROSTATE-Repeatability-Radiomics\statsUtils\metrics.py:98: RuntimeWarning: invalid value encountered in subtract
  return np.sum(np.square(self.pairwiseMean - self.mean)) / (self.n - 1)
D:\Dev\PCrash\QIN-PROSTATE-Repeatability-Radiomics\statsUtils\metrics.py:102: RuntimeWarning: invalid value encountered in subtract
  return np.sum(np.square(self.data1 - self.pairwiseMean) + np.square(self.data2 - self.pairwiseMean)) / (2 * self.n)
D:\Dev\PCrash\QIN-PROSTATE-Repeatability-Radiomics\statsUt

## General Convenience Functions for Plotting

Small helper function to reduce a list to unique values while keeping the order of the elements (keep first)

In [7]:
def unique_keep_order(seq):
  seen = set()
  seen_add = seen.add  # this is for speed optimization, resolving a local variable is cheaper
  return [x for x in seq if not (x in seen or seen_add(x))]

A small helper class for mapping mapping keys to values in a user defined dictionary or returning the key if as value if the key is not defined

In [8]:
class Mapper(object):
  def __init__(self, customDict):
    self._customDict = customDict

  def lookup(self, key):
    if key in self._customDict:
      return self._customDict[key]
    else:
      return key
    

Simple map from indices to letters for subfigures

In [9]:
FIGURE_SUBIDX_MAP = {0: "a", 1: "b", 2: "c", 3: "d"}

#### Bokeh Customizable Scatterplot
This function defines a Bokeh Scatterplot that can be configured to encode different parameters in color and glyph shape.
Currently only supports 1 source for color and 1 or 2 sources for glyph shape. Also currently limited to a maximum of 4 glyph shapes (i.e. this function is not super generalized yet, just enough for the purposes of this analysis, I should think about cleaning this up and puttin git into statsUtils).

In [21]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool, Span
from bokeh.plotting import figure, output_file, show, save, reset_output
from bokeh.io import output_notebook, export_png
from bokeh.colors import RGB
from bokeh.palettes import Category20, Category10
from bokeh.transform import dodge, jitter

from bokeh.models import HoverTool, PanTool, WheelZoomTool, BoxZoomTool, ResetTool, TapTool

def showBokehPlot(dfSelectedMetrics = None, plotWhat = "icc",
                  glyphColorSrc = "binwidth", glyphShapeSrc = ["dim", "normalized"], 
                  colorLegendLookupDict = {}, shapeLegendLookupDict = {},
                  outFilePrefix = "", outFileSuffix = "", writeToFile = False,
                  includeFeatGroupNameInLegend = False,
                  userSingleGlyphSelection = -1):
  
  print("BOKEH DEACTIVATED, REMOVE RETURN FROM CODE")
  return

  cMapper = Mapper(colorLegendLookupDict)
  sMapper = Mapper(shapeLegendLookupDict)
  if isinstance(glyphShapeSrc, str):
    glyphShapeSrc = [glyphShapeSrc]
  
  reset_output()
  output_notebook()

  # add a unique color for each value of the designated source for the glyph color
  colormap = {}
  requiredNoColors = len(dfSelectedMetrics[glyphColorSrc].unique())
  colorScheme = Category10[10]
  if requiredNoColors > 10:
    colorScheme = Category20[20]
  for i, f in enumerate(np.sort(dfSelectedMetrics[glyphColorSrc].unique())):
    colormap[f] = colorScheme[i]
  dfSelectedMetrics["color"] = dfSelectedMetrics.apply(lambda row: colormap[row[glyphColorSrc]], axis = 1)
  
  # add legend for glyphs
  if len(glyphShapeSrc) == 1:
    dfSelectedMetrics["legend"] = dfSelectedMetrics.apply(lambda row: str(sMapper.lookup(row[glyphShapeSrc[0]])) + " " + 
                                                                      str(cMapper.lookup(row[glyphColorSrc])), 
                                                          axis = 1)
  else:
    dfSelectedMetrics["legend"] = dfSelectedMetrics.apply(lambda row: str(sMapper.lookup(row[glyphShapeSrc[0]])) + " " + 
                                                                      str(sMapper.lookup(row[glyphShapeSrc[1]]))  + " " +
                                                                      str(cMapper.lookup(row[glyphColorSrc])), 
                                                          axis = 1)

  # declare data sources, we need seperate  sources for each glyph shape and glyphColor
  sources = []
  if len(glyphShapeSrc) == 1:  
    shapeSource0Values = np.sort(dfSelectedMetrics[glyphShapeSrc[0]].unique()).tolist()
    colorSource0Values = np.sort(dfSelectedMetrics[glyphColorSrc].unique()).tolist()
    for ss0 in shapeSource0Values:
      colorList = []
      for cs0 in colorSource0Values:
        c = ColumnDataSource(data = dfSelectedMetrics[(dfSelectedMetrics[glyphShapeSrc[0]] == ss0) &
                                                      (dfSelectedMetrics[glyphColorSrc] == cs0)])
        colorList.append(c)
      sources.append(colorList)
  else:
    shapeSource0Values = np.sort(dfSelectedMetrics[glyphShapeSrc[0]].unique()).tolist()
    shapeSource1Values = np.sort(dfSelectedMetrics[glyphShapeSrc[1]].unique()).tolist()
    for ss0 in shapeSource0Values:
      for ss1 in shapeSource1Values:
        source = ColumnDataSource(data = dfSelectedMetrics[(dfSelectedMetrics[glyphShapeSrc[0]] == ss0) &
                                                           (dfSelectedMetrics[glyphShapeSrc[1]] == ss1)])
        sources.append(source)
    
    
  # create the plot
  hover = HoverTool(tooltips = [("rc", "@rc"),
                                ("rcp", "@rcp"),
                                ("icc", "@icc"),
                                ("madp", "@meanOfAbsDiffPercent"),
                                ("mean", "@mean"),
                                ("std", "@std"),
                                ("type", "@ctype"),
                                ("structure", "@structure"),
                                ("filter", "@filter"),
                                ("featName", "@featureName")])
  wZoom = WheelZoomTool()
  bZoom = BoxZoomTool()
  reset = ResetTool()
  pan = PanTool()
  
  titleStr = "Legend Code: " + " ".join(glyphShapeSrc) + " " + glyphColorSrc + "     (" + outFileSuffix + ")"
  
  # this is a "trick" to get only the base feature names as y tick labels, but keep the feature groups together
  yRange = np.sort(dfSelectedMetrics["featureGroupAndBaseName"].unique()).tolist()
  if not includeFeatGroupNameInLegend:
    yRange = [yl.split("_")[-1] for yl in yRange]
  
  p = figure(x_range = [-0.6, 1], #x_range=[min(-1, np.min(dfSelectedMetrics[plotWhat].values)),max(1, np.max(dfSelectedMetrics[plotWhat].values))], 
             y_range = yRange, 
             x_axis_label = plotWhat.upper(),
             y_axis_label = "Feature Name",
             tools = [hover, wZoom, bZoom, reset, pan], 
             title = None,#titleStr, 
             plot_width=1200, plot_height=(145 + (76 * len(yRange))),
             min_border_left = 255
            )
    
  # add a reference line where the volume ICC is
  volumeIcc = Span(location=dfSelectedMetrics[(dfSelectedMetrics["featureBaseName"] == "Volume")][plotWhat].tolist()[0],
                   dimension='height', line_color='gray',
                   line_dash='dashed', line_width=2)
  p.add_layout(volumeIcc)
  
  if includeFeatGroupNameInLegend:
    yLabel = 'featureGroupAndBaseName'
  else:
    yLabel = 'featureBaseName'
  

  if (userSingleGlyphSelection < 0) or (len(sources) > 1):
    if len(sources) > 1:
      for i, s in enumerate(sources[1]):
        p.square(plotWhat, dodge(yLabel, 0.05+0.08*i, range=p.y_range), source = s, color = "color", legend = "legend", size = 18, line_width = 2, fill_alpha = 0.5)
        p.segment(x0="iccConfIntLow", y0=dodge(yLabel, 0.05+0.08*i, range=p.y_range), x1="iccConfIntUp",
                  y1=dodge(yLabel, 0.05+0.08*i, range=p.y_range), source = s, color="color", line_width=2, line_alpha = 0.8)
    if len(sources) > 0:
      for i, s in enumerate(sources[0]):
        p.asterisk(plotWhat, dodge(yLabel, -0.3+0.08*i, range=p.y_range), source = s, color = "color", legend = "legend", size = 22, line_width = 2)
        p.segment(x0="iccConfIntLow", y0=dodge(yLabel, -0.3+0.08*i, range=p.y_range), x1="iccConfIntUp",
                  y1=dodge(yLabel, -0.3+0.08*i, range=p.y_range), source = s, color="color", line_width=2, line_alpha = 0.8)
    if len(sources) > 2:  
      for s in sources[2]:
        p.diamond(plotWhat, yLabel, source = s, color = "color", legend = "legend", size = 22, line_width = 2, fill_alpha = 0.5)
    if len(sources) > 3:  
      for s in sources[3]:
        p.circle(plotWhat, yLabel, source = s, color = "color", legend = "legend", size = 18, line_width = 2, fill_alpha = 0.5)
  else:
    if userSingleGlyphSelection == 1:
      p.square(plotWhat, yLabel, source = sources[0], color = "color", legend = "legend", size = 18, line_width = 2, fill_alpha = 0.5)
    if userSingleGlyphSelection == 0:
      p.asterisk(plotWhat, yLabel, source = sources[0], color = "color", legend = "legend", size = 22, line_width = 2)
    if userSingleGlyphSelection == 2:  
      p.diamond(plotWhat, yLabel, source = sources[0], color = "color", legend = "legend", size = 22, line_width = 2, fill_alpha = 0.5)
    if userSingleGlyphSelection == 3:  
      p.circle(plotWhat, yLabel, source = sources[0], color = "color", legend = "legend", size = 18, line_width = 2, fill_alpha = 0.5)
    
  
  p.legend.location = "top_left"
  p.legend.background_fill_color = "snow"
  p.legend.background_fill_alpha = 0.9
  p.legend.glyph_height = 38
  p.legend.glyph_width = 38
  p.legend.padding = 5
  p.legend.spacing = -12

  #p.toolbar.active_scroll = wZoom
  p.toolbar.logo = None
  p.toolbar_location = None
  
  # changing default sizes of fonts
  p.xaxis.axis_label_text_font_size = "16pt"
  p.xaxis.major_label_text_font_size = "14pt"
  p.yaxis.axis_label_text_font_size = "16pt"
  p.yaxis.major_label_text_font_size = "14pt"
  p.legend.label_text_font_size = "14pt"

  
  show(p)

  if writeToFile:
    plotOutBaseFileName = outFilePrefix + plotWhat + "__" + "_".join(glyphShapeSrc) + "_" + glyphColorSrc + "__" + outFileSuffix
    reset_output()
    output_file(os.path.join(os.getcwd(), "EvalData", "plots", plotOutBaseFileName + ".html"))
    #save(p) #this line saves the plot in html
    export_png(p, filename = os.path.join(os.getcwd(), "EvalData", "plots", plotOutBaseFileName + ".png"))

## Figures 6-8
### ICC Bin-width and Normlization on Selected Features (Texture)
ICC for selected features computed on different ROIs on ADC, SUB, and T2w images. Texture features are computed in 2D. Colors represent the bin width for the texture computations, glyph shape represents if the image was normalized or not. No filtering was applied to the image. Dashed line indicates the reference Volume ICC.

In [13]:
selectedFeatureNames = ["glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]


# Select image type: choose from ["ADC1400", "SUB", "T2AX"]
for i, SELECTED_TYPE in enumerate(["ADC1400", "SUB", "T2AX"], 6):
  BASE_IMAGE_TYPE = "BC" if SELECTED_TYPE == "T2AX" else "ORIG"
  # Select segmented structures: choose from ["TumorROI_PZ_1", "PeripheralZone", "WholeGland"]
  for j, SELECTED_STRUCT in enumerate(["TumorROI_PZ_1", "PeripheralZone", "WholeGland"]):
    for DIM in [2]:
      dfSelectedMetrics = dfMetrics[(dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                                    (dfMetrics.icc.notnull()) &
                                    (dfMetrics["filter"] == "original") &
                                    (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                                    (dfMetrics["ctype"] == SELECTED_TYPE) &
                                    (dfMetrics["structure"] == SELECTED_STRUCT) &
                                    (dfMetrics["dim"] == DIM)
                                   ]
      dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "binwidth", ascending = True)
      if dfSelectedMetrics.shape[0] == 0:
        continue
      myColorDict = {5: "binw-5", 10: "binw-10", 15: "binw-15", 20: "binw-20", 40: "binw-40"}
      myShapeDict = {True: "Normed", False: "Original"}
      showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
                    glyphColorSrc = "binwidth", glyphShapeSrc = ["normalized"], 
                    colorLegendLookupDict = myColorDict, shapeLegendLookupDict = myShapeDict,
                    outFilePrefix = "Fig" + str(i) + FIGURE_SUBIDX_MAP[j] + "__",
                    outFileSuffix = "__Texture__Imge_Type_Struct_Dim_Tex=" + BASE_IMAGE_TYPE + "_" + SELECTED_TYPE + "_" + SELECTED_STRUCT + "_" + str(DIM),
                    includeFeatGroupNameInLegend = True,
                    writeToFile = True)
      #display(dfSelectedMetrics[["featureName", "icc", "iccConfIntLow", "iccConfIntUp"]])

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

### ICC Bin-width and Normlization on Selected Features (Firstorder)
ICC for selected features computed on different ROIs on ADC, SUB, and T2w images. No filtering was applied to the image. Dashed line indicates the reference Volume ICC.

In [20]:
selectedFeatureNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", 
                        "firstorder_Variance", "firstorder_Kurtosis", "firstorder_Skewness", "shape_Volume"]

# Select image type: choose from ["ADC1400", "SUB", "T2AX"]
for i, SELECTED_TYPE in enumerate(["ADC1400", "SUB", "T2AX"], 6):
  BASE_IMAGE_TYPE = "BC" if SELECTED_TYPE == "T2AX" else "ORIG"
  # Select segmented structures: choose from ["TumorROI_PZ_1", "PeripheralZone", "WholeGland"]
  for j, SELECTED_STRUCT in enumerate(["TumorROI_PZ_1", "PeripheralZone", "WholeGland"]):
    for DIM in [2]:
      dfSelectedMetrics = dfMetrics[(dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                                    (dfMetrics.icc.notnull()) &
                                    (dfMetrics["filter"] == "original") &
                                    (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                                    (dfMetrics["ctype"] == SELECTED_TYPE) &
                                    (dfMetrics["structure"] == SELECTED_STRUCT) &
                                    (dfMetrics["dim"] == DIM) & 
                                    (dfMetrics["binwidth"] == 10)
                                   ]
      dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "binwidth", ascending = True)
      if dfSelectedMetrics.shape[0] == 0:
        continue
      myColorDict = {True: "", False: ""}
      myShapeDict = {True: "Normed", False: "Original"}
      showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
                    glyphColorSrc = "normalized", glyphShapeSrc = ["normalized"], 
                    colorLegendLookupDict = myColorDict, shapeLegendLookupDict = myShapeDict,
                    outFilePrefix = "Fig" + str(i) + FIGURE_SUBIDX_MAP[j] + "__",
                    outFileSuffix = "__Firstorder__Imge_Type_Struct_Dim_Tex=" + BASE_IMAGE_TYPE + "_" + SELECTED_TYPE + "_" + SELECTED_STRUCT + "_" + str(DIM),
                    includeFeatGroupNameInLegend = True,
                    writeToFile = True)
      #display(dfSelectedMetrics[["featureName", "icc", "iccConfIntLow", "iccConfIntUp"]])

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## *Figure not in paper: Comparing bias corrected vs original T2AX*

In [ ]:
for SELECTED_TYPE in ["T2AX"]:
  # Select segmented structures: choose from ["WholeGland", "PeripheralZone", "TumorROI_PZ_1", "NormalROI_PZ_1"]
  for SELECTED_STRUCT in ["WholeGland", "PeripheralZone", "TumorROI_PZ_1"]:
    for DIM in [2]:
      dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) &
                                    (dfMetrics["filter"] == "original") &
                                    (dfMetrics["baseImageType"].isin(["ORIG", "BC"])) &
                                    (dfMetrics["ctype"] == SELECTED_TYPE) &
                                    (dfMetrics["structure"] == SELECTED_STRUCT) &
                                    (dfMetrics["dim"] == DIM) & 
                                    (dfMetrics["normalized"] == False) &
                                    (dfMetrics["binwidth"] == 15)
                                   ]
      dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "binwidth", ascending = True)
      if dfSelectedMetrics.shape[0] == 0:
        continue
      myColorDict = {"BC": "", "ORIG": ""}
      myShapeDict = {"BC": "Bias-corrected", "ORIG": "Original"}
      showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
                    glyphColorSrc = "baseImageType", glyphShapeSrc = ["baseImageType"], 
                    colorLegendLookupDict = myColorDict, shapeLegendLookupDict = myShapeDict,
                    outFileSuffix = "Imge_Type_Struct_Dim=" + SELECTED_TYPE + "_" + SELECTED_STRUCT + "_" + str(DIM),
                    includeFeatGroupNameInLegend = True,
                    writeToFile = False)

## Figures 9 and 10 
KDE plots and Rank Count for bin width influence
### Preparation for 9a and 10a

In [ ]:
BASE_IMAGE_TYPE = "ORIG_or_BC"
selectedFeatureNames = ["glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) &
                              (((dfMetrics["baseImageType"] == "ORIG") & (dfMetrics["ctype"] != "T2AX")) |
                               ((dfMetrics["baseImageType"] == "BC") & (dfMetrics["ctype"] == "T2AX"))) &
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["filter"] == "original")][['binwidth',
                                                                    'ctype',
                                                                    'dim',
                                                                    'featureName',
                                                                    'featureGroup',
                                                                    'filter',
                                                                    'icc',
                                                                    'normalized',
                                                                    'structure'
                                                                   ]]

dfSelectedMetrics = dfSelectedMetrics[(dfSelectedMetrics["featureGroup"] != "firstorder") & 
                                      (dfSelectedMetrics["featureGroup"] != "shape")]

dfGroup = dfSelectedMetrics.groupby(['ctype','dim','featureName','featureGroup','filter','normalized','structure'])
dfSelectedMetrics["rank"] = dfGroup["icc"].rank(ascending=False, method='min').astype(int)

### Figure 9a

In [ ]:
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 18}
plt.rc('font', **font)

fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)

maxIccDiffPerGroup = dfGroup["icc"].agg(['max','min'])
maxIccDiffPerGroup['maxDiff'] = maxIccDiffPerGroup['max']-maxIccDiffPerGroup['min']
plot = sb.kdeplot(maxIccDiffPerGroup['maxDiff'], shade = True)
plot.set(xlim = (-0.15,1.5))
fig.savefig(os.path.join(os.getcwd(), "EvalData", "plots", "Fig9a__" + BASE_IMAGE_TYPE + "_bin__maxdiff_kdeplot__select_glcm.png"), bbox_inches='tight')

### Figure 10a

In [ ]:
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 18}
plt.rc('font', **font)

fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sb.countplot(x="binwidth", hue="rank", data=dfSelectedMetrics)
fig.savefig(os.path.join(os.getcwd(), "EvalData", "plots", "Fig10a__" + BASE_IMAGE_TYPE + "_bin__rank_countplot__select_glcm.png"), bbox_inches='tight')

### Preparation for 9b and 10b

In [ ]:
BASE_IMAGE_TYPE = "ORIG_or_BC"
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) &
                              (((dfMetrics["baseImageType"] == "ORIG") & (dfMetrics["ctype"] == "ADC1400_OFF")) |
                               ((dfMetrics["baseImageType"] == "BC") & (dfMetrics["ctype"] == "T2AX")))][['binwidth',
                                                                                                          'ctype',
                                                                                                          'dim',
                                                                                                          'featureName',
                                                                                                          'featureGroup',
                                                                                                          'filter',
                                                                                                          'icc',
                                                                                                          'normalized',
                                                                                                          'structure'
                                                                                                         ]]

dfSelectedMetrics = dfSelectedMetrics[(dfSelectedMetrics["featureGroup"] != "firstorder") & 
                                      (dfSelectedMetrics["featureGroup"] != "shape")]

dfGroup = dfSelectedMetrics.groupby(['ctype','dim','featureName','featureGroup','filter','normalized','structure'])
dfSelectedMetrics["rank"] = dfGroup["icc"].rank(ascending=False, method='min').astype(int)

### Figure 9b

In [ ]:
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 18}
plt.rc('font', **font)

fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)

maxIccDiffPerGroup = dfGroup["icc"].agg(['max','min'])
maxIccDiffPerGroup['maxDiff'] = maxIccDiffPerGroup['max']-maxIccDiffPerGroup['min']
plot = sb.kdeplot(maxIccDiffPerGroup['maxDiff'], shade = True)
plot.set(xlim = (-0.15,1.5))
fig.savefig(os.path.join(os.getcwd(), "EvalData", "plots", "Fig9b__" + BASE_IMAGE_TYPE + "_bin__maxdiff_kdeplot.png"), bbox_inches='tight')

### Figure 10b

In [ ]:
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 18}
plt.rc('font', **font)

fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sb.countplot(x="binwidth", hue="rank", data=dfSelectedMetrics)
fig.savefig(os.path.join(os.getcwd(), "EvalData", "plots", "Fig10b__" + BASE_IMAGE_TYPE + "_bin__rank_countplot.png"), bbox_inches='tight')

## Figure 11
ICC for selected features computed on the Tumor ROI on T2w images with (a) 2D and (b) 3D wavelet pre-filtering and feature extraction.
### Figure 11a

In [ ]:
selectedFeatureNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", 
                        "firstorder_Variance", "firstorder_Kurtosis", "firstorder_Skewness",
                        "glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
BASE_IMAGE_TYPE = "BC"
BIN = 15
DIM = 2
NORM = False
CTYPE = "T2AX"
STRUCTURE = "TumorROI_PZ_1"
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                              (dfMetrics["binwidth"] == BIN) &
                              (dfMetrics["dim"] == DIM) &
                              (dfMetrics["normalized"] == NORM) &
                              (dfMetrics["ctype"] == CTYPE) &
                              (dfMetrics["structure"] == STRUCTURE) &
                              ((dfMetrics["featureGroup"] == "shape") | (dfMetrics["featureGroup"] == "firstorder") 
                                                                      | (dfMetrics["featureGroup"] == "glcm")) &
                              ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "wavelet-LL")
                                                                   | (dfMetrics["filter"] == "wavelet-LH")
                                                                   | (dfMetrics["filter"] == "wavelet-HL")
                                                                   | (dfMetrics["filter"] == "wavelet-HH"))
                             ]
dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "filter")

myShapeDict = {True: "", False: ""}
showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
              glyphColorSrc = "filter", glyphShapeSrc = "normalized",
              shapeLegendLookupDict = myShapeDict,
              outFilePrefix = "Fig11a__",
              outFileSuffix = "Wavelet__Bin_Dim_Normalized_CType_Struct=" + str(BIN) + "_" + str(DIM) + "_" + str(NORM) + "_" + str(CTYPE) + "_" + str(STRUCTURE),
              writeToFile = True)

### Figure 11b

In [ ]:
selectedFeatureNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", 
                        "firstorder_Variance", "firstorder_Kurtosis", "firstorder_Skewness",
                        "glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
BASE_IMAGE_TYPE = "BC"
BIN = 15
DIM = 3
NORM = False
CTYPE = "T2AX"
STRUCTURE = "TumorROI_PZ_1"
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                              (dfMetrics["binwidth"] == BIN) &
                              (dfMetrics["dim"] == DIM) &
                              (dfMetrics["normalized"] == NORM) &
                              (dfMetrics["ctype"] == CTYPE) &
                              (dfMetrics["structure"] == STRUCTURE) &
                              ((dfMetrics["featureGroup"] == "shape") | (dfMetrics["featureGroup"] == "firstorder") 
                                                                      | (dfMetrics["featureGroup"] == "glcm")) &
                              ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "wavelet-LLL")
                                                                   | (dfMetrics["filter"] == "wavelet-HLL")
                                                                   | (dfMetrics["filter"] == "wavelet-LHL")
                                                                   | (dfMetrics["filter"] == "wavelet-LLH")
                                                                   | (dfMetrics["filter"] == "wavelet-LHH")
                                                                   | (dfMetrics["filter"] == "wavelet-HLH")
                                                                   | (dfMetrics["filter"] == "wavelet-HHL")
                                                                   | (dfMetrics["filter"] == "wavelet-HHH"))
                             ]
dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "filter")

myShapeDict = {True: "", False: ""}
showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
              glyphColorSrc = "filter", glyphShapeSrc = "normalized",
              shapeLegendLookupDict = myShapeDict,
              outFilePrefix = "Fig11b__",
              outFileSuffix = "Wavelet__Bin_Dim_Normalized_CType_Struct=" + str(BIN) + "_" + str(DIM) + "_" + str(NORM) + "_" + str(CTYPE) + "_" + str(STRUCTURE),
              writeToFile = True, userSingleGlyphSelection = 1)

## Figure 12
ICC for selected features computed on the Tumor ROI on T2w images with 3D LoG pre-filtering and texture features extracted in 2D (a) and 3D (b). 
### Figure 12a

In [ ]:
selectedFeatureNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", 
                        "firstorder_Variance", "firstorder_Kurtosis", "firstorder_Skewness",
                        "glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
BASE_IMAGE_TYPE = "BC"
BIN = 15
NORM = False
CTYPE = "T2AX"
STRUCTURE = "TumorROI_PZ_1"
DIM = 2
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                              (dfMetrics["binwidth"] == BIN) &
                              (dfMetrics["normalized"] == NORM) &
                              (dfMetrics["ctype"] == CTYPE) &
                              (dfMetrics["structure"] == STRUCTURE) &
                              (dfMetrics["dim"] == DIM) &
                              ((dfMetrics["featureGroup"] == "shape") | (dfMetrics["featureGroup"] == "firstorder") 
                                                                      | (dfMetrics["featureGroup"] == "glcm")) &
                              ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "log-sigma-2-0-mm-3D")
                                                                   | (dfMetrics["filter"] == "log-sigma-3-0-mm-3D")
                                                                   | (dfMetrics["filter"] == "log-sigma-4-0-mm-3D")
                                                                   | (dfMetrics["filter"] == "log-sigma-5-0-mm-3D"))
                             ]
dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "filter")

myColorDict = {}
myColorDict = {"log-sigma-2-0-mm-3D": "log-sigma-2.0mm", 
               "log-sigma-3-0-mm-3D": "log-sigma-3.0mm", 
               "log-sigma-4-0-mm-3D": "log-sigma-4.0mm", 
               "log-sigma-5-0-mm-3D": "log-sigma-5.0mm"}
myShapeDict = {2: "", 3: ""}
showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
              glyphColorSrc = "filter", glyphShapeSrc = "dim",
              colorLegendLookupDict = myColorDict, shapeLegendLookupDict = myShapeDict,
              outFilePrefix = "Fig12a__",
              outFileSuffix = "Log__Bin_Normalized_Ctype_Struct_Dim=" + str(BIN) + "_" + str(NORM) + "_" + str(CTYPE) + "_" + str(STRUCTURE) + "_" + str(DIM),
              writeToFile = True)

### Figure 12b

In [ ]:
selectedFeatureNames = ["glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
BASE_IMAGE_TYPE = "BC"
BIN = 15
NORM = False
CTYPE = "T2AX"
STRUCTURE = "TumorROI_PZ_1"
DIM = 3
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                              (dfMetrics["binwidth"] == BIN) &
                              (dfMetrics["normalized"] == NORM) &
                              (dfMetrics["ctype"] == CTYPE) &
                              (dfMetrics["structure"] == STRUCTURE) &
                              (dfMetrics["dim"] == DIM) &
                              ((dfMetrics["featureGroup"] == "shape") | (dfMetrics["featureGroup"] == "firstorder") 
                                                                      | (dfMetrics["featureGroup"] == "glcm")) &
                              ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "log-sigma-2-0-mm-3D")
                                                                   | (dfMetrics["filter"] == "log-sigma-3-0-mm-3D")
                                                                   | (dfMetrics["filter"] == "log-sigma-4-0-mm-3D")
                                                                   | (dfMetrics["filter"] == "log-sigma-5-0-mm-3D"))
                             ]
dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "filter")

myColorDict = {}
myColorDict = {"log-sigma-2-0-mm-3D": "log-sigma-2.0mm", 
               "log-sigma-3-0-mm-3D": "log-sigma-3.0mm", 
               "log-sigma-4-0-mm-3D": "log-sigma-4.0mm", 
               "log-sigma-5-0-mm-3D": "log-sigma-5.0mm"}
myShapeDict = {2: "", 3: ""}
showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
              glyphColorSrc = "filter", glyphShapeSrc = "dim",
              colorLegendLookupDict = myColorDict, shapeLegendLookupDict = myShapeDict,
              outFilePrefix = "Fig12b__",
              outFileSuffix = "Log__Bin_Normalized_Ctype_Struct_Dim=" + str(BIN) + "_" + str(NORM) + "_" + str(CTYPE) + "_" + str(STRUCTURE) + "_" + str(DIM),
              writeToFile = True, userSingleGlyphSelection = 1)

## Figure 13
ICC for selected features computed on the Tumor ROI on T2w images with single pixel pre-filtering and texture features extracted in 2D (a) and 3D (b)
### Figure 13a 

In [ ]:
selectedFeatureNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", 
                        "firstorder_Variance", "firstorder_Kurtosis", "firstorder_Skewness",
                        "glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
BASE_IMAGE_TYPE = "BC"
BIN = 15
NORM = False
CTYPE = "T2AX"
STRUCTURE = "TumorROI_PZ_1"
DIM = 2
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                              (dfMetrics["binwidth"] == BIN) &
                              (dfMetrics["normalized"] == NORM) &
                              (dfMetrics["ctype"] == CTYPE) &
                              (dfMetrics["structure"] == STRUCTURE) &
                              (dfMetrics["dim"] == DIM) &
                              ((dfMetrics["featureGroup"] == "shape") | (dfMetrics["featureGroup"] == "firstorder") 
                                                                      | (dfMetrics["featureGroup"] == "glcm")) &
                              ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "square")
                                                                   | (dfMetrics["filter"] == "squareroot")
                                                                   | (dfMetrics["filter"] == "exponential")
                                                                   | (dfMetrics["filter"] == "logarithm"))
                             ]
dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "filter")

myShapeDict = {2: "", 3: ""}
showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
              glyphColorSrc = "filter", glyphShapeSrc = "dim",
              shapeLegendLookupDict = myShapeDict,
              outFilePrefix = "Fig13a__",
              outFileSuffix = "ExpEtc__Bin_Normalized_Ctype_Struct_Dim=" + str(BIN) + "_" + str(NORM) + "_" + str(CTYPE) + "_" + str(STRUCTURE) + "_" + str(DIM),
              writeToFile = True)

### Figure 13b

In [ ]:
selectedFeatureNames = ["glcm_JointEnergy","glcm_JointEntropy", "glcm_Idm", "glcm_Correlation", 
                        "glcm_Contrast", "shape_Volume"]
BASE_IMAGE_TYPE = "BC"
BIN = 15
NORM = False
CTYPE = "T2AX"
STRUCTURE = "TumorROI_PZ_1"
DIM = 3
dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                              (dfMetrics["featureGroupAndBaseName"].isin(selectedFeatureNames)) &
                              (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                              (dfMetrics["binwidth"] == BIN) &
                              (dfMetrics["normalized"] == NORM) &
                              (dfMetrics["ctype"] == CTYPE) &
                              (dfMetrics["structure"] == STRUCTURE) &
                              (dfMetrics["dim"] == DIM) &
                              ((dfMetrics["featureGroup"] == "shape") | (dfMetrics["featureGroup"] == "firstorder") 
                                                                      | (dfMetrics["featureGroup"] == "glcm")) &
                              ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "square")
                                                                   | (dfMetrics["filter"] == "squareroot")
                                                                   | (dfMetrics["filter"] == "exponential")
                                                                   | (dfMetrics["filter"] == "logarithm"))
                             ]
dfSelectedMetrics = dfSelectedMetrics.sort_values(by = "filter")

myShapeDict = {2: "", 3: ""}
showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
              glyphColorSrc = "filter", glyphShapeSrc = "dim",
              shapeLegendLookupDict = myShapeDict,
              outFilePrefix = "Fig13b__",
              outFileSuffix = "ExpEtc__Bin_Normalized_Ctype_Struct_Dim=" + str(BIN) + "_" + str(NORM) + "_" + str(CTYPE) + "_" + str(STRUCTURE) + "_" + str(DIM),
              writeToFile = True, userSingleGlyphSelection = 1)

## Convenience function for Top3 Plots

In [26]:
def getTop3FeaturesPerGroupAndBaseImage(bin_width, dim, norm, ctype, structure, 
                                        base_image_types, 
                                        groups = ["shape", "firstorder", "glcm", "glrlm", "glszm"]):
  featuresNames = []
  for GROUP in groups:
    for BASE_IMAGE_TYPE in base_image_types: # "ORIG", "BC", "BC+MRN", "BC+REG"
      dfSelectedMetrics = dfSortedMetrics[(dfSortedMetrics.icc.notnull()) &
                                          (dfMetrics["baseImageType"] == BASE_IMAGE_TYPE) &
                                          (dfSortedMetrics["binwidth"] == bin_width) &
                                          (dfSortedMetrics["dim"] == dim) &
                                          (dfSortedMetrics["normalized"] == norm) &
                                          (dfSortedMetrics["ctype"] == ctype) &
                                          (dfSortedMetrics["structure"] == structure) &
                                          (dfSortedMetrics["featureGroup"] == GROUP)
                                         ]
      dfSelectedMetrics = dfSelectedMetrics.drop_duplicates(subset = "featureGroupAndBaseName", keep = "first")
      featuresNames.extend((dfSelectedMetrics["featureName"].values.tolist())[:3])
  return featuresNames


def getAllMetrics(bin_width, dim, norm, ctype, structure, 
                  featureGroupAndBaseNames,
                  base_image_types, 
                  groups = ["shape", "firstorder", "glcm", "glrlm", "glszm"]):
  dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) & 
                                (dfMetrics["baseImageType"].isin(base_image_types)) &
                                (dfMetrics["binwidth"] == bin_width) &
                                (dfMetrics["dim"] == dim) &
                                (dfMetrics["normalized"] == norm) &
                                (dfMetrics["ctype"] == ctype) &
                                (dfMetrics["structure"] == structure) &
                                ((dfMetrics["featureGroupAndBaseName"].isin(featureGroupAndBaseNames)) | 
                                 (dfMetrics["featureBaseName"] == "Volume")) &
                                (dfMetrics["featureGroup"].isin(groups)) &
                                ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "wavelet-LL")
                                                                     | (dfMetrics["filter"] == "wavelet-LH")
                                                                     | (dfMetrics["filter"] == "wavelet-HL")
                                                                     | (dfMetrics["filter"] == "wavelet-HH")
                                                                     | ((dfMetrics["filter"] == "wavelet-LLL") & 
                                                                        (dfMetrics["dim"] == 3))
                                                                     | (dfMetrics["filter"] == "wavelet-HLL")
                                                                     | (dfMetrics["filter"] == "wavelet-LHL")
                                                                     | (dfMetrics["filter"] == "wavelet-LLH")
                                                                     | (dfMetrics["filter"] == "wavelet-LHH")
                                                                     | (dfMetrics["filter"] == "wavelet-HLH")
                                                                     | (dfMetrics["filter"] == "wavelet-HHL")
                                                                     | (dfMetrics["filter"] == "wavelet-HHH")
                                                                     | (dfMetrics["filter"] == "log-sigma-2-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "log-sigma-3-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "log-sigma-4-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "log-sigma-5-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "square")
                                                                     | (dfMetrics["filter"] == "squareroot")
                                                                     | (dfMetrics["filter"] == "exponential")
                                                                     | (dfMetrics["filter"] == "logarithm"))
                               ]
  return dfSelectedMetrics


def top3Plot(base_image_type, bin_width, dim, norm, ctype, structure, outFilePrefix = ""):
  featuresNames = getTop3FeaturesPerGroupAndBaseImage(bin_width, dim, norm, ctype, structure, [base_image_type])
  featureGroupAndBaseNames = [f.split("_", maxsplit = 1)[1] for f in featuresNames]
  dfSelectedMetrics = getAllMetrics(bin_width, dim, norm, ctype, structure, featureGroupAndBaseNames, [base_image_type])

  dfSelectedMetrics = dfSelectedMetrics.sort_values(by = ["filterGroup", "filter"])
  myShapeDict = {"a_original": "", "log": "", "singlepix": "", "wavelet": ""}
  showBokehPlot(dfSelectedMetrics, plotWhat = "icc",
                glyphColorSrc = "filter", glyphShapeSrc = "filterGroup",
                shapeLegendLookupDict = myShapeDict,
                outFilePrefix = outFilePrefix,
                outFileSuffix = "Top3_AllFilters__Bin_Dim_Normalized_CType_Struct=" + str(bin_width) + "_" + str(dim) + "_" + str(norm) + "_" + str(ctype) + "_" + str(structure),
                writeToFile = True,
                includeFeatGroupNameInLegend = True)
  outputTable = dfSelectedMetrics[["featureGroupAndBaseName", "filter", "icc", "iccConfIntLow", "iccConfIntUp"]]
  outputTable = outputTable.sort_values(by=['featureGroupAndBaseName', 'icc'], ascending=False)
  outputTableFilename = os.path.join(os.getcwd(), "EvalData", "plots", outFilePrefix + "Top3_AllFilters__Bin_Dim_Normalized_CType_Struct=" + str(bin_width) + "_" + str(dim) + "_" + str(norm) + "_" + str(ctype) + "_" + str(structure) + ".html")
  outputTable.to_html(outputTableFilename)
  display(outputTable)

## Figure 14
Top 3 features for each feature group by ICC in T2w images for (a) the Tumor ROI and (b) the Peripheral Zone
### Figure 14a

In [27]:
top3Plot("BC", 15, 2, False, "T2AX", "TumorROI_PZ_1", "Fig14a__")

D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BOKEH DEACTIVATED, REMOVE RETURN FROM CODE


,featureGroupAndBaseName,filter,icc,iccConfIntLow,iccConfIntUp
324941,shape_Volume,original,0.856503,0.638581,0.948816
324592,shape_SurfaceVolumeRatio,original,0.896062,0.729461,0.963428
325866,shape_Maximum3DDiameter,original,0.883667,0.700338,0.958892
324510,shape_MajorAxis,original,0.880965,0.694071,0.957898
325191,glszm_ZoneVariance,wavelet-HH,0.977755,0.937803,0.992385
325507,glszm_ZoneVariance,log-sigma-3-0-mm-3D,0.956523,0.880739,0.985012
325532,glszm_ZoneVariance,square,0.949862,0.863278,0.982677
325235,glszm_ZoneVariance,exponential,0.937492,0.831394,0.978313
325008,glszm_ZoneVariance,wavelet-LH,0.872192,0.673910,0.954659
324907,glszm_ZoneVariance,wavelet-LL,0.835532,0.592747,0.940906


### Figure 14b

In [28]:
top3Plot("BC", 15, 2, False, "T2AX", "PeripheralZone", "Fig14b__")

D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BOKEH DEACTIVATED, REMOVE RETURN FROM CODE


,featureGroupAndBaseName,filter,icc,iccConfIntLow,iccConfIntUp
251837,shape_Volume,original,0.859408,0.690175,0.940431
252762,shape_Maximum3DDiameter,original,0.959717,0.905063,0.983447
252338,shape_Maximum2DDiameterSlice,original,0.968011,0.924173,0.986887
252643,shape_Maximum2DDiameterColumn,original,0.979345,0.950651,0.991562
252068,glszm_ZoneEntropy,wavelet-LH,0.921880,0.820590,0.967529
251455,glszm_ZoneEntropy,wavelet-HH,0.880695,0.733412,0.949782
251991,glszm_ZoneEntropy,wavelet-LL,0.720940,0.435622,0.876460
252393,glszm_ZoneEntropy,wavelet-HL,0.716014,0.427336,0.874079
252021,glszm_ZoneEntropy,log-sigma-4-0-mm-3D,0.699224,0.399451,0.865904
251765,glszm_ZoneEntropy,log-sigma-5-0-mm-3D,0.694686,0.392008,0.863679


## Figure 15
Top 3 features for each feature group by ICC in ADC images for (a) the Tumor ROI and (b) the Peripheral Zone
### Figure 15a

In [29]:
top3Plot("ORIG", 15, 2, True, "ADC1400", "TumorROI_PZ_1", "Fig15a__")

D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BOKEH DEACTIVATED, REMOVE RETURN FROM CODE


,featureGroupAndBaseName,filter,icc,iccConfIntLow,iccConfIntUp
55370,shape_Volume,original,0.693552,0.293211,0.890936
55021,shape_SurfaceVolumeRatio,original,0.749193,0.395600,0.912588
54901,shape_Maximum2DDiameterRow,original,0.607183,0.150600,0.855449
55971,glszm_SmallAreaHighGrayLevelEmphasis,log-sigma-3-0-mm-3D,0.977049,0.932251,0.992627
56139,glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LL,0.876671,0.668766,0.958965
55307,glszm_SmallAreaHighGrayLevelEmphasis,original,0.833115,0.568660,0.943600
55411,glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LH,0.749008,0.395244,0.912518
55332,glszm_SmallAreaHighGrayLevelEmphasis,squareroot,0.675988,0.262691,0.883909
55473,glszm_SmallAreaHighGrayLevelEmphasis,square,0.637831,0.199120,0.868314
55897,glszm_SmallAreaHighGrayLevelEmphasis,logarithm,0.576235,0.103741,0.842139


### Figure 15b

In [30]:
top3Plot("ORIG", 15, 2, True, "ADC1400", "PeripheralZone", "Fig15b__")

D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
D:\miniconda3\envs\QINProst\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BOKEH DEACTIVATED, REMOVE RETURN FROM CODE


,featureGroupAndBaseName,filter,icc,iccConfIntLow,iccConfIntUp
31002,shape_Volume,original,0.853308,0.668818,0.939872
31927,shape_Maximum3DDiameter,original,0.898460,0.763509,0.958969
31503,shape_Maximum2DDiameterSlice,original,0.875517,0.714657,0.949334
31808,shape_Maximum2DDiameterColumn,original,0.899818,0.766450,0.959535
30527,glszm_ZonePercentage,original,0.976002,0.940899,0.990533
31566,glszm_ZonePercentage,wavelet-LL,0.975920,0.940700,0.990501
31076,glszm_ZonePercentage,wavelet-HL,0.973863,0.935733,0.989683
31446,glszm_ZonePercentage,square,0.964221,0.912648,0.985835
31608,glszm_ZonePercentage,log-sigma-4-0-mm-3D,0.960616,0.904101,0.984390
31789,glszm_ZonePercentage,wavelet-LH,0.952468,0.884946,0.981113


## Convenience Function for AboveVolumeICC Countplots

In [ ]:
def plotAboveVolumeIccCount(base_image_type, bin_width, dim, norm, ctype, structure, outFilePrefix = ""):
  dfSelectedMetrics = dfMetrics[(dfMetrics.icc.notnull()) &
                                (dfMetrics["baseImageType"] == base_image_type) &
                                (dfMetrics["binwidth"] == bin_width) &
                                (dfMetrics["dim"] == dim) &
                                (dfMetrics["normalized"] == norm) &
                                (dfMetrics["ctype"] == ctype) &
                                (dfMetrics["structure"] == structure) &
                                ((dfMetrics["filter"] == "original") | (dfMetrics["filter"] == "wavelet-LL")
                                                                     | (dfMetrics["filter"] == "wavelet-LH")
                                                                     | (dfMetrics["filter"] == "wavelet-HL")
                                                                     | (dfMetrics["filter"] == "wavelet-HH")
                                                                     | ((dfMetrics["filter"] == "wavelet-LLL") & (dfMetrics["dim"] == 3))
                                                                     | (dfMetrics["filter"] == "wavelet-HLL")
                                                                     | (dfMetrics["filter"] == "wavelet-LHL")
                                                                     | (dfMetrics["filter"] == "wavelet-LLH")
                                                                     | (dfMetrics["filter"] == "wavelet-LHH")
                                                                     | (dfMetrics["filter"] == "wavelet-HLH")
                                                                     | (dfMetrics["filter"] == "wavelet-HHL")
                                                                     | (dfMetrics["filter"] == "wavelet-HHH")
                                                                     | (dfMetrics["filter"] == "log-sigma-2-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "log-sigma-3-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "log-sigma-4-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "log-sigma-5-0-mm-3D")
                                                                     | (dfMetrics["filter"] == "square")
                                                                     | (dfMetrics["filter"] == "squareroot")
                                                                     | (dfMetrics["filter"] == "exponential")
                                                                     | (dfMetrics["filter"] == "logarithm"))
                               ]
  volumeIcc = dfSelectedMetrics[(dfSelectedMetrics["featureBaseName"] == "Volume")]["icc"].values[0]
  dfSelectedMetrics = dfSelectedMetrics[(dfSelectedMetrics["icc"] > volumeIcc)]
  dfSelectedMetrics = dfSelectedMetrics.sort_values(by = ["filterGroup", "filter"])
  dfFeatures = dfSelectedMetrics["featureGroupAndBaseName"].unique()
  noFeaturesAboveVolIcc = len(dfFeatures)

  sb.set(style="ticks", font_scale=2.0)
  fig, ax = plt.subplots()
  fig.set_size_inches(11.7, 8.27)
  g = sb.countplot(x="filter", data=dfSelectedMetrics)
  for item in g.get_xticklabels():
      item.set_rotation(-90)
  plt.plot([-1, 14], [noFeaturesAboveVolIcc, noFeaturesAboveVolIcc], linewidth=2, linestyle='--', color='gray')
  fig.savefig(os.path.join(os.getcwd(), "EvalData", "plots", outFilePrefix + "aboveVolumeIcc_filter_countplot_" + ctype + "_"+ structure + ".png"), bbox_inches='tight')


## Figure 16
Overview of how often the particular pre-filters appear among the features which reach an ICC higher than Volume on T2w images for (a) Tumor ROI and (b) Peripheral Zone.
### Figure 16a

In [ ]:
plotAboveVolumeIccCount("BC", 15, 2, False, "T2AX", "TumorROI_PZ_1", "Fig16a__")

### Figure 16b

In [ ]:
plotAboveVolumeIccCount("BC", 15, 2, False, "T2AX", "PeripheralZone", "Fig16b__")

## Figure 17
Overview of how often the particular pre-filters appear among the features which reach an ICC higher than Volume on ADC images for (a) Tumor ROI and (b) Peripheral Zone.
### Figure 17a

In [ ]:
plotAboveVolumeIccCount("ORIG", 15, 2, True, "ADC1400", "TumorROI_PZ_1", "Fig17a__")

### Figure 17b

In [ ]:
plotAboveVolumeIccCount("ORIG", 15, 2, True, "ADC1400", "PeripheralZone", "Fig17b__")

## Convenience Functions for Trend Line Plots
Also uses some convenience functions from the Top3 plots, so they need to be executed for this.

In [ ]:
def plotTrendLines(left_image_type, right_image_type, bin_width, dim, norm, ctype, structure, featuresNames, outFilePrefix = ""):
  postfix = "_Selected"
  if not featuresNames or len(featuresNames) == 0:
    postfix = "_Top3"
    featuresNames = getTop3FeaturesPerGroupAndBaseImage(bin_width, dim, norm, ctype, structure, [left_image_type, right_image_type])
  featureGroupAndBaseNames = [f.split("_", maxsplit = 1)[1] for f in featuresNames]
  featureGroupAndBaseNames = unique_keep_order(featureGroupAndBaseNames)
  
  dfSelectedMetrics = getAllMetrics(bin_width, dim, norm, ctype, structure, featureGroupAndBaseNames, [left_image_type, right_image_type])

  # Map from codes to human readable text
  xLabelMap = {
    "ORIG": "Original",
    "BC": "T2w", # Bias Corrected
    "BC+MRN": "T2w Normalized by Reference", # Bias Corrected + Muscle Ref Norm
    "BC+REG": "T2w with Registration" # Bias Corrected + Registration
  }
  
  dfSelectedMetricsLeft = dfSelectedMetrics[(dfSelectedMetrics["baseImageType"] == left_image_type)]
  dfSelectedMetricsLeft = dfSelectedMetricsLeft.sort_values(by = "icc", ascending = False)
  dfSelectedMetricsLeft = dfSelectedMetricsLeft[(dfSelectedMetricsLeft["featureName"].isin(featuresNames))]
  #dfSelectedMetricsLeft = dfSelectedMetricsLeft.drop_duplicates(subset = "featureGroupAndBaseName", keep = "first")
  
  dfSelectedMetricsRight = dfSelectedMetrics[(dfSelectedMetrics["baseImageType"] == right_image_type)]
  dfSelectedMetricsRight = dfSelectedMetricsRight.sort_values(by = "icc", ascending = False)
  dfSelectedMetricsRight = dfSelectedMetricsRight[(dfSelectedMetricsRight["featureName"].isin(featuresNames))]
  #dfSelectedMetricsRight = dfSelectedMetricsRight.drop_duplicates(subset = "featureGroupAndBaseName", keep = "first")

  font = {'family' : 'sans',
          'weight' : 'normal',
          'size'   : 18}
  colorMap = {"firstorder": "C0",
              "glcm"      : "C1",
              "glrlm"     : "C2",
              "glszm"     : "C3",
              "shape"     : "C4"}

  matplotlib.style.use("default")
  plt.rc('font', **font)
  plt.figure(figsize=(10,10))
  plt.ylabel('$\it{ICC}$', fontsize = 20)

  textY = 1.02
  iccAndFilters = dfSelectedMetricsRight[["icc", "filter", "featureName", "featureGroup"]]
  for idx, row in iccAndFilters.iterrows():
    group = row["featureGroup"]
    y2 = row["icc"]
    y1 = dfSelectedMetricsLeft[(dfSelectedMetricsLeft["featureName"] == row["featureName"])
                              ]["icc"].values[0]
    plt.plot([1, 2], [y1, y2], linestyle='-', color=colorMap[group])

    plt.text(2.3, textY, row["featureName"], fontsize=14, horizontalalignment='left', verticalalignment='center', color=colorMap[group])
    ll = plt.plot([2.01, 2.28], [y2, textY], linestyle=':', color=colorMap[group]) # returns a sequence of line objects 
    ll[0].set_clip_on(False)
    textY -= 0.05

    plt.axis([0.95,2.05,-0.2,1.05]) 


  plt.xticks([1,2], [xLabelMap[left_image_type], xLabelMap[right_image_type]])

  plt.savefig(os.path.join(os.getcwd(), "EvalData", "plots", outFilePrefix + "TrendLines_" + STRUCTURE + "_" + CTYPE + "_" + left_image_type + "_vs_" + right_image_type + postfix + ".png"), bbox_inches='tight')


## Figure 18 a + b
Change in Tumor ROI ICCs from T2w to T2w normalized by a reference region (muscle) for (a) literature recommended features, (b) 3 most stable features from each of the feature classes.

In [ ]:
# Hard define features, REMOVE if I want to use the top 3 features
featuresNames = ["original_firstorder_Mean", "original_firstorder_Median", "original_firstorder_10Percentile", "original_firstorder_Variance",
                 "original_firstorder_Kurtosis", "original_firstorder_Skewness", "original_glcm_JointEnergy","original_glcm_JointEntropy",
                 "original_glcm_Idm", "original_glcm_Correlation", "original_glcm_Contrast", "original_shape_Volume"]

plotTrendLines("BC", "BC+MRN", 15, 2, False, "T2AX", "TumorROI_PZ_1", featuresNames, "Fig18a__")
plotTrendLines("BC", "BC+MRN", 15, 2, False, "T2AX", "TumorROI_PZ_1", None, "Fig18b__")


## Figure 19 a + b
Change in ICCs from T2w Tumor ROIs to registered T2w Tumor ROIs for (a) literature recommended features, (b) top 3 features.

In [ ]:
# Hard define features, REMOVE if I want to use the top 3 features
featuresNames = ["original_firstorder_Mean", "original_firstorder_Median", "original_firstorder_10Percentile", "original_firstorder_Variance",
                 "original_firstorder_Kurtosis", "original_firstorder_Skewness", "original_glcm_JointEnergy","original_glcm_JointEntropy",
                 "original_glcm_Idm", "original_glcm_Correlation", "original_glcm_Contrast", "original_shape_Volume"]

plotTrendLines("BC", "BC+REG", 15, 2, False, "T2AX", "TumorROI_PZ_1", featuresNames, "Fig19a__")
plotTrendLines("BC", "BC+REG", 15, 2, False, "T2AX", "TumorROI_PZ_1", None, "Fig19b__")


## Convenience Function for Plotting Paired Cases Plot

In [ ]:
def pairedCasesPlot(ctype, structure, feature, imageType, normalized):
  BIN = 15
  DIM = 2
  CTYPE = ctype  # ADC1400 T2AX
  STRUCTURE = structure # TumorROI_PZ_1 PeripheralZone
  FEATURE = feature    # "original_firstorder_Mean"

  dfSelectedFeatures = dfFeaturesJointTP[(dfFeaturesJointTP[FEATURE+"_t1"].notnull()) &
                                         (dfFeaturesJointTP[FEATURE+"_t2"].notnull()) &
                                         (dfFeaturesJointTP["binwidth"] == BIN) &
                                         (dfFeaturesJointTP["dim"] == DIM) &
                                         (dfFeaturesJointTP["canonicalType"] == CTYPE) &
                                         (dfFeaturesJointTP["segmentedStructure"] == STRUCTURE)]

  # Select from "ORIG", "BC", "BC+MRN"
  imageTypeLabelMap = {
    "ORIG": "Original",
    "BC": "Bias Corrected",
    "BC+MRN": "Bias Corrected + Muscle Ref Norm",
    "BC+REG": "Bias Corrected + Registration"
  }
  IMAGE_TYPE = imageType
  NORM = normalized

  dfSelectedFeatures = dfSelectedFeatures[(dfSelectedFeatures["baseImageType"] == IMAGE_TYPE) &
                                          (dfSelectedFeatures["normalized"] == NORM)]

  patNoLookup = { "PCAMPMRI-00775": 1  ,
                  "PCAMPMRI-00784": 2  ,
                  "PCAMPMRI-00812": 3  ,
                  "PCAMPMRI-00821": 4  ,
                  "PCAMPMRI-00824": 5  ,
                  "PCAMPMRI-00838": 6  ,
                  "PCAMPMRI-00840": 7  ,
                  "PCAMPMRI-00936": 8  ,
                  "PCAMPMRI-00945": 9  ,
                  "PCAMPMRI-00949": 10 ,
                  "PCAMPMRI-00967": 11 ,
                  "PCAMPMRI-00990": 12 ,
                  "PCAMPMRI-01020": 13 ,
                  "PCAMPMRI-01030": 14 ,
                  "PCAMPMRI-01033": 15 }

  patientNames = dfSelectedFeatures["patient"].values
  fTp1 = dfSelectedFeatures[FEATURE+"_t1"].values
  fTp2 = dfSelectedFeatures[FEATURE+"_t2"].values
  n = fTp1.shape[0]
  overallMean = np.mean([fTp1, fTp2])
  pairwiseMean = np.mean([fTp1, fTp2], axis=0)
  brms = np.sqrt(np.sum(np.square(pairwiseMean - overallMean)) / (n - 1))
  wrms = np.sqrt(np.sum(np.square(fTp1 - pairwiseMean) + np.square(fTp2 - pairwiseMean)) / (2 * n))
  #print(bms, wms)
  featValues = []
  featValues.append(fTp1)
  featValues.append(pairwiseMean)
  featValues.append(fTp2)
  #featValues

  plt.figure(figsize=(16,9))
  sb.swarmplot(data=featValues, orient="h")
  for i in range(len(featValues[0])):
    plt.plot([featValues[0][i], featValues[2][i]], [0, 2], linestyle=':', color="gray")
    plt.text(featValues[0][i], -0.05, str(patNoLookup[patientNames[i]]), fontsize=10, horizontalalignment='center', verticalalignment='bottom', color="gray")
    plt.text(featValues[2][i], 2.05, str(patNoLookup[patientNames[i]]), fontsize=10, horizontalalignment='center', verticalalignment='top', color="gray")

  plt.plot([overallMean, overallMean], [-0.2, 2.2], linestyle='-', color="gray")
  plt.text(overallMean, -0.3, "Overall Mean", fontsize=14, horizontalalignment='center', verticalalignment='center', color="gray")

  plt.plot([overallMean - (0.5 * brms), overallMean + (0.5 * brms)], [2.3, 2.3], linestyle='-', color="orange")
  plt.plot([overallMean - (0.5 * wrms), overallMean + (0.5 * wrms)], [2.4, 2.4], linestyle='-', color="gray")
  x = overallMean + (0.6 * max(wrms,brms))
  plt.text(x, 2.3, "between cases rmse", fontsize=14, horizontalalignment='left', verticalalignment='center', color="orange")
  plt.text(x, 2.4, "within cases rmse", fontsize=14, horizontalalignment='left', verticalalignment='center', color="gray")
  plt.title(CTYPE + " " + STRUCTURE + " " + FEATURE + " " + IMAGE_TYPE + "-normalized=" + str(NORM))



  plt.yticks([0,1,2], ["TP 1", "PairMean", "TP 2"])
  #plt.savefig(os.path.join(os.getcwd(), "EvalData", "plots", "PairedCases_" + CTYPE + "_" + STRUCTURE + "_" + FEATURE + "_" + IMAGE_TYPE + "-normed-" + str(NORM) + ".png"), bbox_inches='tight')
  #plt.show()

## Paired Cases Plots (not used in Paper)

In [ ]:
pairedCasesPlot("ADC1400", "PeripheralZone", "original_firstorder_Mean", "ORIG", False)
pairedCasesPlot("ADC1400", "PeripheralZone", "original_firstorder_Mean", "ORIG", True)
pairedCasesPlot("ADC1400", "PeripheralZone", "original_glcm_Contrast", "ORIG", False)
pairedCasesPlot("ADC1400", "PeripheralZone", "original_glcm_Contrast", "ORIG", True)
pairedCasesPlot("ADC1400", "PeripheralZone", "original_glcm_Idm", "ORIG", False)
pairedCasesPlot("ADC1400", "PeripheralZone", "original_glcm_Idm", "ORIG", True)

pairedCasesPlot("ADC1400", "TumorROI_PZ_1", "original_firstorder_Mean", "ORIG", False)
pairedCasesPlot("ADC1400", "TumorROI_PZ_1", "original_firstorder_Mean", "ORIG", True)
pairedCasesPlot("ADC1400", "TumorROI_PZ_1", "original_glcm_Idm", "ORIG", False)
pairedCasesPlot("ADC1400", "TumorROI_PZ_1", "original_glcm_Idm", "ORIG", True)

pairedCasesPlot("T2AX", "PeripheralZone", "original_firstorder_Mean", "ORIG", False)
pairedCasesPlot("T2AX", "PeripheralZone", "original_firstorder_Mean", "ORIG", True)

pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_firstorder_Mean", "ORIG", False)
pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_firstorder_Mean", "ORIG", True)

pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_glcm_Contrast", "BC+MRN", False)
pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_glcm_Contrast", "BC+REG", False)
pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_glcm_Contrast", "BC", False)
pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_glcm_Contrast", "ORIG", False)
pairedCasesPlot("T2AX", "TumorROI_PZ_1", "original_glcm_Contrast", "ORIG", True)



## Correlation Matrix Plot (not used in Paper)

In [ ]:
BIN = 15
DIM = 2
CTYPE = "T2AX"  # ADC1400 T2AX
STRUCTURE = "PeripheralZone" # TumorROI_PZ_1 PeripheralZone
IMAGE_TYPE = "BC"
NORM = True
FILTER = "original"

dfSelectedFeatures = dfFeaturesJointTP[(dfFeaturesJointTP["binwidth"] == BIN) &
                                       (dfFeaturesJointTP["dim"] == DIM) &
                                       (dfFeaturesJointTP["canonicalType"] == CTYPE) &
                                       (dfFeaturesJointTP["segmentedStructure"] == STRUCTURE) &
                                       (dfFeaturesJointTP["baseImageType"] == IMAGE_TYPE) &
                                       (dfFeaturesJointTP["normalized"] == NORM)]



featuresNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", "firstorder_Variance",
                 "firstorder_Kurtosis", "firstorder_Skewness", "glcm_JointEnergy","glcm_JointEntropy",
                 "glcm_Idm", "glcm_Correlation", "glcm_Contrast", "shape_Volume"]
featuresNames = ["firstorder_Mean", "firstorder_Median", "firstorder_10Percentile", "firstorder_Variance",
                 "firstorder_Kurtosis", "firstorder_Skewness"]
featuresNames2 = ["glcm_JointEnergy","glcm_JointEntropy",
                 "glcm_Idm", "glcm_Correlation", "glcm_Contrast", "shape_Volume"]

featuresNames = [FILTER + "_" + f for f in featuresNames]
ft1 = [f + "_t1" for f in featuresNames]
ft2 = [f + "_t2" for f in featuresNames]
featuresNames = ft1 + ft2

corr = dfSelectedFeatures[featuresNames].corr()

fig, ax = plt.subplots()
fig.set_size_inches(16, 16)
sb.heatmap(corr, annot=True, linewidths=.5)